# 📊 Preparação de Dados - Previsão de Vendas

Este notebook realiza a preparação e limpeza dos dados para o modelo de previsão de vendas por PDV/SKU.

## 🎯 Objetivos
- Unificar datasets de transações, lojas, produtos e informações auxiliares
- Limpar e tratar valores inconsistentes
- Criar features de engenharia (sazonalidade, histórico)
- Preparar dados para modelagem

## 📋 Etapas do Pipeline
1. **Importação de bibliotecas**
2. **Leitura dos datasets**
3. **Unificação dos dados**
4. **Limpeza e tratamento**
5. **Engenharia de features**
6. **Exportação dos dados processados**

## 📚 Importação de Bibliotecas

Carregamento das bibliotecas necessárias para manipulação e processamento dos dados:
- **Polars**: Biblioteca de alta performance para manipulação de DataFrames
- **NumPy**: Operações numéricas e matemáticas

In [1]:
import polars as pl
import numpy as np

## 📂 Leitura dos Datasets

Carregamento dos datasets brutos necessários para a análise:

### Datasets Principais:
- **df_store**: Informações dos pontos de venda (PDVs)
- **df_transaction**: Histórico de transações de vendas
- **df_product**: Catálogo de produtos e suas características
- **df_zipcode**: Dados geográficos por CEP
- **df_holiday**: Calendário de feriados para análise de sazonalidade

> **Nota**: Os arquivos parquet oferecem melhor performance de leitura e compressão comparado a CSVs.

In [2]:
df_store = pl.read_parquet('../../data/raw/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet')
df_transaction = pl.read_parquet('../../data/raw/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet')
df_product = pl.read_parquet('../../data/raw/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet')
df_zipcode = pl.read_csv('../../data/raw/georef-zipcode.csv', separator=';')
df_holiday = pl.read_csv('../../data/processed/processed_usa_holiday.csv', separator=',')

## 🔗 Unificação dos Datasets

Processo de join entre os diferentes datasets para criar uma visão unificada dos dados:

### Estratégia de Join:
1. **Transações + Lojas**: Join por `internal_store_id` ↔ `pdv`
2. **Resultado + Produtos**: Join por `internal_product_id` ↔ `produto`

### Tipo de Join:
- **Left Join**: Mantém todas as transações, mesmo que não tenham informações completas de loja/produto
- Garante que nenhuma transação seja perdida no processo

In [3]:
df = df_transaction.join(
    df_store,
    left_on="internal_store_id",
    right_on="pdv",
    how='left'
).join(
    df_product,
    left_on="internal_product_id",
    right_on="produto",
    how='left'
)

## 🧹 Limpeza e Tratamento dos Dados

Etapas essenciais para garantir a qualidade dos dados antes da modelagem.

### 🎯 Tratamento da Variável Target (Quantity)

**Problema**: Valores negativos e decimais na quantidade vendida
**Solução**: 
- Remover transações com quantidade negativa (devoluções/erros)
- Arredondar valores decimais para inteiros (unidades de produtos)

**Justificativa**: Quantidades negativas podem representar devoluções ou erros de sistema, que não são úteis para prever vendas futuras.

In [4]:
df = df.filter(df['quantity'] >= 0)
df = df.with_columns(df['quantity'].round().cast(pl.Int64))

### 📈 Tratamento de Outliers Extremos

**Problema Identificado**: Data 2022-09-11 apresenta valores anômalos/outliers extremos
**Impacto**: Outliers podem distorcer o treinamento do modelo
**Estratégia**: Investigar e tratar valores anômalos dessa data específica

> **Importante**: Outliers podem ser eventos promocionais legítimos ou erros de sistema. A análise deve distinguir entre ambos.

In [5]:
# Remove from day 2022-09-11 the products with quantity greater than the percentile 75
# and products appearing for the first time

outlier_date = pl.lit('2022-09-11').str.to_date()

outlier_products = df.filter(
    pl.col('transaction_date') == outlier_date
)['internal_product_id'].unique()

normal_products = df.filter(
    (pl.col('internal_product_id').is_in(outlier_products)) &
    (pl.col('transaction_date') < outlier_date)
)['internal_product_id'].unique()

# Calculate the 75th percentile of quantity for the outlier date
outlier_percentile_75 = df.filter(
    pl.col('transaction_date') == outlier_date
).select(
    pl.col('quantity').quantile(0.75, interpolation='linear')
).item()

# Filter the DataFrame
df = df.filter(
    (pl.col('transaction_date') != outlier_date) |
    (
        pl.col('quantity').le(outlier_percentile_75) &
        pl.col('internal_product_id').is_in(normal_products)
    )
)

### 📅 Transformação de Features Temporais

**Objetivo**: Extrair componentes temporais relevantes para capturar sazonalidade

**Features Criadas**:
- **month**: Mês da transação (1-12) - sazonalidade mensal
- **week_of_year**: Semana do ano (0-53) - sazonalidade semanal

**Importância**: Padrões sazonais são cruciais para previsão de vendas no varejo.

In [6]:
df = df.with_columns([
    pl.col('transaction_date').dt.month().alias('month'),
    pl.col('transaction_date').dt.strftime("%U").cast(pl.Int64).alias('week_of_year')
])

## ⚡ Engenharia de Features

Processo de criação de features avançadas para melhorar a performance do modelo.

### 🛠️ Configuração de Variáveis para Feature Engineering

**Definição de Grupos de Variáveis**:
- **cols**: Métricas numéricas (quantidade, valores, lucro)
- **keys**: Variáveis categóricas e temporais
- **Partições específicas**: Agrupamentos por cidade, PDV, produto

**Estratégia**: Diferentes combinações de chaves para criar features agregadas específicas por contexto.

In [7]:
cols = ['quantity','gross_value','net_value','gross_profit','discount']
keys = [
    'internal_product_id', 'internal_store_id', 'distributor_id',
    'premise', 'categoria_pdv', 'zipcode', 'tipos', 'label', 'subcategoria',
    'marca', 'fabricante', 'month', 'week_of_year', 'city'
]
city_partition = ['internal_product_id', 'city']
city_month_keys = ['internal_product_id', 'city', 'month']
city_week_keys = ['internal_product_id', 'city', 'week_of_year']
pdv_week_keys = ['internal_product_id', 'internal_store_id', 'week_of_year']
product_city_partition = ['internal_product_id', 'city']
product_pdv_partition = ['internal_product_id', 'internal_store_id']

### 🗺️ Enriquecimento Geográfico

**Processo**: Adicionar informação de cidade através do CEP
**Join**: `df` + `df_zipcode` por `zipcode`
**Resultado**: Feature geográfica `city` para análise regional

**Valor**: Permite capturar padrões de vendas por região geográfica.

In [8]:
df_zipcode = df_zipcode.rename({'Zip Code': 'zipcode', 'Official USPS city name': 'city'})
df = df.join(df_zipcode.select(['zipcode', 'city']), on='zipcode', how='left')

### 🎉 Tratamento de Feriados

**Processo**: Criar feature binária para identificar feriados
**Transformação**: 
- Converter string de data para formato date
- Criar flag `holiday = 1` para datas de feriado

**Importância**: Feriados têm impacto significativo nos padrões de vendas do varejo.

In [9]:
df_holiday = df_holiday.with_columns([
    pl.col('Date').str.to_date().alias('Date'),
    pl.lit(1).alias('holiday')
])

df = df.join(
    df_holiday.select(['Date', 'holiday']),
    left_on='transaction_date',
    right_on='Date',
    how='left'
)

df = df.with_columns(
    pl.col('holiday').fill_null(0)
)

### 🗑️ Remoção de Colunas Desnecessárias

**Colunas Removidas**:
- `taxes`: Não relevante para previsão de quantidade
- `categoria`: Redundante com outras categorizações
- `descricao`: Texto livre, dificulta modelagem
- `reference_date`, `transaction_date`: Já extraímos features temporais

**Objetivo**: Simplificar dataset mantendo apenas features relevantes.

In [ ]:
df = df.drop(['taxes','categoria','descricao','reference_date', 'transaction_date'])

### 📊 Features de Histórico Mensal

**Objetivo**: Criar features de vendas do mês anterior por produto/cidade
**Agregações**: Soma mensal de quantidade, valores, lucro e desconto
**Lag**: Shift de 1 mês para capturar tendência histórica

**Features Criadas**:
- `previous_month_quantity_sum`: Quantidade vendida no mês anterior
- `previous_month_gross_value_sum`: Valor bruto do mês anterior
- `previous_month_net_value_sum`: Valor líquido do mês anterior
- E outras métricas mensais históricas

**Importância**: Histórico recente é forte preditor de vendas futuras.

In [10]:
monthly_aggs = []
for c in cols:
    monthly_aggs += [
        pl.col(c).sum().alias(f"monthly_{c}_sum"),
    ]

monthly_totals = df.group_by(city_month_keys).agg(monthly_aggs)

monthly_shifts = []
monthly_shifts_names = []
for c in cols:
    monthly_shifts += [
        pl.col(f"monthly_{c}_sum").shift(n=1).over(product_city_partition).alias(f"previous_month_{c}_sum"),
    ]
    monthly_shifts_names += [
        f"previous_month_{c}_sum",
    ]

previous_month_values = monthly_totals.sort(city_month_keys).with_columns(monthly_shifts)

### 📈 Features de Lag Semanal

**Objetivo**: Capturar padrão semanal de vendas por produto/PDV
**Lags Criados**: 5 semanas anteriores (`quantity_lag1` a `quantity_lag5`)
**Granularidade**: Por produto + PDV específico

**Uso**: 
- Detectar tendências de curto prazo
- Capturar sazonalidade semanal
- Identificar produtos em alta/baixa

**Valor**: Lags de curto prazo são essenciais para capturar momentum de vendas.

In [18]:
quantity_totals = df.group_by(pdv_week_keys).agg(
    pl.col('quantity').sum().alias('quantity')
)

quantity_shifts = [
    pl.col("quantity").shift(n=i).over(
        product_pdv_partition
    ).alias(f"quantity_lag{i}") for i in range(1, 6)
]
quantity_shifts_names = [
    f"quantity_lag{i}" for i in range(1, 6)
]

previous_quantity_values = quantity_totals.sort(pdv_week_keys).with_columns(quantity_shifts)

### 🔗 Consolidação Final dos Dados

**Processo**:
1. **Agregação Principal**: Agrupar por todas as chaves categóricas e temporais
2. **Join Histórico Mensal**: Adicionar features do mês anterior
3. **Join Lags Semanais**: Adicionar lags de quantidade semanal
4. **Features Derivadas**: Calcular métricas financeiras (discount_rate, profit_margin)

**Features Derivadas Criadas**:
- `discount_rate_month`: Taxa de desconto do mês anterior
- `profit_margin_month`: Margem de lucro do mês anterior

**Resultado**: Dataset final pronto para modelagem com todas as features de engenharia.

In [19]:
df = df.group_by(keys).agg([
    pl.col("quantity").sum().alias("quantity"),
    pl.col("holiday").max().alias("holiday")
]).join(
    previous_month_values.select(city_month_keys + monthly_shifts_names),
    on=city_month_keys,
    how="left"
).join(
    previous_quantity_values.select(pdv_week_keys + quantity_shifts_names),
    on=pdv_week_keys,
    how="left"
).with_columns([
    (pl.col('previous_month_discount_sum') / pl.col('previous_month_gross_value_sum')).fill_null(0).replace([np.inf, -np.inf], 0).alias('discount_rate_month'),
    (pl.col('previous_month_gross_profit_sum') / pl.col('previous_month_gross_value_sum')).fill_null(0).replace([np.inf, -np.inf], 0).alias('profit_margin_month')
])

## 💾 Salvando Dados Processados

**Formato**: Parquet para otimização de performance
**Localização**: `../../data/processed/processed_data.parquet`
**Conteúdo**: Dataset final com todas as transformações e features de engenharia

**Próximos Passos**: Os dados processados estão prontos para serem utilizados nos notebooks de modelagem (`02_models/`).

In [26]:
df.write_parquet('../../data/processed/processed_data.parquet')